In [5]:
!pip install onnxruntime
!sudo apt-get update -y
!sudo apt-get install python3.9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Python 3.8.10
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,000 kB]

In [37]:
import os
# create input dir
if not os.path.exists('images'):
    os.mkdir('images')
# create output dir
if not os.path.exists('uploads'):
    os.mkdir('uploads')

In [53]:
import cv2
import onnxruntime
import numpy as np
import sys
from pathlib import Path
from numpy import floating


def LoadImageAsNDArray(path) :
    image_mat = cv2.imread(path)
    image, width_and_height = PreprocessImage(image_mat)
    image = np.asarray(np.expand_dims(image, 0))
    return (image, tuple(width_and_height))

def PreprocessImage(image, x32=True) :
    height, width = image.shape[:2]
    if x32:
        def to_32s(x):
            if x < 256:
                return 256
            return x - x % 32
        image = cv2.resize(image, (to_32s(width), to_32s(height)))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32) / 127.5 - 1.0
    return (image, [width, height])

def SaveImage(transform_image_ndarray, width_and_height, output_image_path):
    transform_image_ndarray = (transform_image_ndarray.squeeze() + 1.0) / 2 * 255
    transform_image_ndarray = transform_image_ndarray.astype(np.uint8)
    transform_image_ndarray = cv2.resize(transform_image_ndarray, (width_and_height[0], width_and_height[1]))
    cv2.imwrite(output_image_path, cv2.cvtColor(transform_image_ndarray, cv2.COLOR_RGB2BGR))

def TransformImage(input_image_path):

    # load onnx runtime model
    print('Loading ONNX runtime model...')
    onnx_path = 'onnx/animeganv3_H64_model.onnx'
    with open(onnx_path, mode='rb') as fp:
        onnx = fp.read()
    session = onnxruntime.InferenceSession(onnx, providers=['CPUExecutionProvider'])
    x = session.get_inputs()[0].name
    y = session.get_outputs()[0].name
    print('Loaded ONNX runtime model.')

    # start inference
    print('Processing...')

    image_ndarray, width_and_height = LoadImageAsNDArray('uploads/'+input_image_path)

    # run inference
    transform_data = session.run(None, {x: image_ndarray})

    # save image
    SaveImage(transform_data[0], width_and_height, 'images/'+input_image_path)
    print(f'Processed image: "{input_image_path}" ({width_and_height[0]}×{width_and_height[1]})')



In [54]:
# the image must be uploaded in the uploads folder 
# change the input with image name
TransformImage('dog1.jpg')
# cv2.imread()

Loading ONNX runtime model...
Loaded ONNX runtime model.
Processing...
Processed image: "dog1.jpg" (640×480)
